# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import numpy as np
import pandas as pd

data_sample = pd.read_csv("/Users/user/Downloads/Sentiment140.csv").sample(500)
data_sample.columns = ['target','id','date','flag','user','text']
data_sample.head()

,target,id,date,flag,user,text
923211,4,1754628204,Sun May 10 06:53:55 PDT 2009,NO_QUERY,ananyah,@Karen230683 well done! I'm proud of you
646123,0,2236440122,Fri Jun 19 03:42:47 PDT 2009,NO_QUERY,sugaredtiffieee,i wish i were taller and wayyyy wayyyy thinner...
107150,0,1823834021,Sat May 16 23:08:05 PDT 2009,NO_QUERY,ohrielley,I miss my beaches
703389,0,2255694118,Sat Jun 20 11:38:56 PDT 2009,NO_QUERY,jamescooley,Can't get Flash to work with FF3.5 on x64 Linu...
14639,0,1553793374,Sat Apr 18 15:13:41 PDT 2009,NO_QUERY,JaneLebak,@stutterdude I never even noticed the ABBA shi...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [2]:
%%time

import functs

data_sample["text"] = data_sample["text"].apply(lambda x: functs.clean_up(x))
data_sample["text_processed"] = data_sample["text"].apply(lambda x: functs.tokenize(x))
data_sample["text_processed"] = data_sample["text_processed"].apply(lambda x: functs.stem_and_lemmatize(x))
data_sample["text_processed"] = data_sample["text_processed"].apply(lambda x: functs.remove_stopwords2(x))
data_sample.head()

CPU times: user 20.2 s, sys: 1.45 s, total: 21.6 s
Wall time: 21.8 s


,target,id,date,flag,user,text,text_processed
923211,4,1754628204,Sun May 10 06:53:55 PDT 2009,NO_QUERY,ananyah,karen well done i m proud of you,[proud]
646123,0,2236440122,Fri Jun 19 03:42:47 PDT 2009,NO_QUERY,sugaredtiffieee,i wish i were taller and wayyyy wayyyy thinner...,"[wish, taller, wayyyi, wayyyi, thinner, short,..."
107150,0,1823834021,Sat May 16 23:08:05 PDT 2009,NO_QUERY,ohrielley,i miss my beaches,"[miss, beach]"
703389,0,2255694118,Sat Jun 20 11:38:56 PDT 2009,NO_QUERY,jamescooley,can t get flash to work with ff on x linux loo...,"[flash, work, ff, x, linux, onli, x, build, av..."
14639,0,1553793374,Sat Apr 18 15:13:41 PDT 2009,NO_QUERY,JaneLebak,stutterdude i never even noticed the abba shir...,"[stutterdud, notic, abba, shirt, lousi]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [3]:
from nltk import FreqDist

In [4]:
corpus = []

[corpus.append(x) for text in data_sample["text_processed"] for x in text]

fdist = FreqDist(corpus)
fdist

FreqDist({'wa': 35, 'love': 34, 'day': 34, 'work': 28, 'today': 23, 'make': 21, 'back': 18, 'realli': 18, 'time': 17, 'feel': 17, ...})

In [5]:
corpus_words = list(fdist)[:500]

In [6]:
import pickle

filename = "corpus.pkl"
with open(filename, "wb") as file:
    pickle.dump(corpus_words, file)

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [7]:
def find_features(doc):
    
    row = [doc]
    [row.append((item in doc)) for item in corpus_words]
    
    return row

In [8]:
col_names = ["doc"]
[col_names.append(w) for w in corpus_words]
corpus_df = pd.DataFrame(columns = col_names)
corpus_df

,doc,wa,love,day,work,today,make,back,realli,time,...,sensit,amanduhmari,rampantheart,prep,nw,bsi,wid,ceh,pst,subway


In [9]:
%%time

for doc in data_sample["text_processed"].iloc[:500]:
    corpus_df.loc[len(corpus_df.index)] = find_features(doc)
    
corpus_df

CPU times: user 9.95 s, sys: 30.4 ms, total: 9.98 s
Wall time: 9.98 s


,doc,wa,love,day,work,today,make,back,realli,time,...,sensit,amanduhmari,rampantheart,prep,nw,bsi,wid,ceh,pst,subway
0,[proud],False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,"[wish, taller, wayyyi, wayyyi, thinner, short,...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"[miss, beach]",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"[flash, work, ff, x, linux, onli, x, build, av...",False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"[stutterdud, notic, abba, shirt, lousi]",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[miss, mad, give, call, miss, realli, call, ho...",False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
496,"[flowersbyfarha, random, stupid, definit, smel...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
497,"[mbeasi, calaggi, found, pack, onc, wa, shelf,...",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
498,"[seubecca, wa, check, regain, contact, sleep, ...",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
corpus_df['sentiment'] = [True if x == 4 else False for x in data_sample['target']]
corpus_df

/var/folders/gj/4j_ddf3s73x6p0hjdxl3kfn80000gp/T/ipykernel_9344/1736712172.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  corpus_df['sentiment'] = [True if x == 4 else False for x in data_sample['target']]


,doc,wa,love,day,work,today,make,back,realli,time,...,amanduhmari,rampantheart,prep,nw,bsi,wid,ceh,pst,subway,sentiment
0,[proud],False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,"[wish, taller, wayyyi, wayyyi, thinner, short,...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"[miss, beach]",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"[flash, work, ff, x, linux, onli, x, build, av...",False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"[stutterdud, notic, abba, shirt, lousi]",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[miss, mad, give, call, miss, realli, call, ho...",False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
496,"[flowersbyfarha, random, stupid, definit, smel...",False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
497,"[mbeasi, calaggi, found, pack, onc, wa, shelf,...",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
498,"[seubecca, wa, check, regain, contact, sleep, ...",True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
corpus_df = corpus_df.reset_index(drop=True)

In [13]:
corpus_df.sentiment.value_counts()

False    265
True     235
Name: sentiment, dtype: int64

In [14]:
corpus_tf = corpus_df.drop(columns='doc')
corpus_tf

,wa,love,day,work,today,make,back,realli,time,feel,...,amanduhmari,rampantheart,prep,nw,bsi,wid,ceh,pst,subway,sentiment
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
497,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
498,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
nltk_list = [(dict(corpus_tf.iloc[x,:-1]),corpus_tf["sentiment"][x]) for x in range(len(corpus_tf))]



### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [16]:
from nltk import NaiveBayesClassifier

train = nltk_list[:16000]
test = nltk_list[16000:]

In [17]:
nbc = NaiveBayesClassifier.train(train)

In [18]:
nbc.show_most_informative_features()

Most Informative Features
                    cool = True             True : False  =      5.6 : 1.0
                     hey = True             True : False  =      5.6 : 1.0
                    feel = True            False : True   =      5.5 : 1.0
                    wish = True            False : True   =      5.0 : 1.0
                   watch = True             True : False  =      4.7 : 1.0
                    amaz = True             True : False  =      4.1 : 1.0
                  awesom = True             True : False  =      3.4 : 1.0
                       p = True             True : False  =      3.4 : 1.0
                   start = True             True : False  =      3.4 : 1.0
                    plea = True            False : True   =      3.3 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [21]:
from nltk.classify import accuracy

accuracy(nbc, test)

#whaattt

0

In [22]:
filename = "nbc.pkl"
with open(filename, "wb") as file:
    pickle.dump(nbc, file)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here